In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import LineString
import sys

sys.path.append('../')
import road
from overpass import fetch_overpass, get_overpass_query
from elevation import get_elevation_from_srtm, calc_incline
from typing import *

In [5]:

columns = ['highway', 'maxspeed', 'lanes', 'name', 'oneway', 'surface']
highway_list = ["motorway", "motorway_link", "trunk", "trunk_link", "primary", "primary_link", 
                  "secondary", "secondary_link", "tertiary", "tertiary_link", "residential","cycleway"]
cycleway_list = None
cycleway_columns = ['cycleway:both', 'cycleway:left','cycleway:right']
if "cycleway" in highway_list:
    cycleway_list = ["lane", "opposite", "opposite_lane", "track", "opposite_track", 
                     "share_busway", "opposite_share_busway", "shared_lane",]
    columns += cycleway_columns
    columns += ['cycleway']
    
    #'oneway:bicycle','cycleway:left:oneway','cycleway:right:oneway',

In [519]:
columns

['highway',
 'maxspeed',
 'lanes',
 'name',
 'oneway',
 'surface',
 'cycleway:both',
 'cycleway:left',
 'cycleway:right',
 'cycleway']

In [520]:
bbox = [45.516012863655845,-73.61165474010419,45.54058887207495,-73.56153948806578]
bbox = [45.45, -73.6492855702676, 45.644709880535515, -73.44752816469271]
bbox = (*bbox,)

In [521]:
overpass_query = get_overpass_query(bbox,highway_list,cycleway_list)

In [522]:
fetch_overpass(overpass_query,columns,'')

OVERPASS Request ...
Convert to GeoPandas ...


/home/boivin/.pyenv/versions/quetzal_env/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


Write (way.geojson) ...


In [27]:
links, nodes = road.get_links_and_nodes('way.geojson', split_direction=True)
nodes = nodes.set_crs(links.crs)

In [524]:
links = rename_bicycle_tags(links,'cycleway')
links = rename_bicycle_tags(links,'cycleway:both')
links = rename_bicycle_tags(links,'cycleway:left')
links = rename_bicycle_tags(links,'cycleway:right')


links['combine_cycle_tag'] = links['cycleway'] +' '+ \
                                links['cycleway:both'] +' '+ \
                                links['cycleway:left'] +' '+ \
                                links['cycleway:right'] 


# simple method. everything with a tag highway is an highway both side. using the road oneway.
bike_dict={}
for string in links['combine_cycle_tag'].unique():
    val = string.split(' ')
    if val[0]=='yes':
        bike_dict[string]='yes'
    elif val[0] == 'shared':
        bike_dict[string]='shared'
    elif 'yes' in val[1:]:
        bike_dict[string]='yes'
    elif  'share' in val[1:]:
        bike_dict[string]='share'
    else :
        bike_dict[string]='no'
links['cycleway'] = links['combine_cycle_tag'].apply(lambda x: bike_dict.get(x))
links.loc[links['highway']=='cycleway','cycleway'] = 'yes'

['no', 'yes', 'shared']
['no', 'yes', 'shared']
['no', 'yes', 'shared']
['no', 'yes', 'shared']


In [525]:
#links.to_file('b_links.geojson',drivers='GeoJSON')
#nodes.to_file('b_nodes.geojson',drivers='GeoJSON')

In [526]:
links = links.drop(columns = cycleway_columns)
#remove highway not asked for. (because of cycleway)
links = links[links['highway'].isin(highway_list)]

In [527]:
links

,id,type,highway,maxspeed,lanes,name,oneway,surface,cycleway,a,b,geometry,combine_cycle_tag
road_link_0,4320462,way,trunk,70,4,Autoroute Ville-Marie,yes,asphalt,no,road_node_12066,road_node_15879,"LINESTRING (-73.55832 45.50704, -73.55806 45.5...",no no no no
road_link_1,4320471,way,residential,30,1,Rue Mackay,yes,asphalt,no,road_node_8702,road_node_8772,"LINESTRING (-73.57878 45.49667, -73.57870 45.4...",no no no no
road_link_2,4320471,way,residential,30,1,Rue Mackay,yes,asphalt,no,road_node_8772,road_node_11514,"LINESTRING (-73.57870 45.49663, -73.57867 45.4...",no no no no
road_link_3,4320471,way,residential,30,1,Rue Mackay,yes,asphalt,no,road_node_11514,road_node_10813,"LINESTRING (-73.57705 45.49584, -73.57696 45.4...",no no no no
road_link_4,4320473,way,tertiary,40,1,Rue du Fort,yes,asphalt,no,road_node_15802,road_node_7236,"LINESTRING (-73.58373 45.49316, -73.58366 45.4...",no no no no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
road_link_60381,1190379608,way,cycleway,NaN,NaN,longueuil ouest-est,no,asphalt,yes,road_node_9384,road_node_12124,"LINESTRING (-73.49963 45.54059, -73.49957 45.5...",yes no no no
road_link_60382,1190379609,way,cycleway,NaN,NaN,longueuil ouest-est,no,asphalt,yes,road_node_25824,road_node_25063,"LINESTRING (-73.49952 45.54068, -73.49749 45.5...",no no no no
road_link_60383,1190379610,way,cycleway,NaN,NaN,longueuil ouest-est,no,asphalt,yes,road_node_25063,road_node_1453,"LINESTRING (-73.49686 45.54346, -73.49682 45.5...",yes no no no
road_link_60384,1190379611,way,cycleway,NaN,NaN,longueuil ouest-est,no,asphalt,yes,road_node_11929,road_node_2694,"LINESTRING (-73.49482 45.54554, -73.49421 45.5...",no no no no


In [540]:
bike_links = links[links['cycleway']!='no'].copy()

bike_links['abset'] = [frozenset(el) for el in zip(bike_links['a'], bike_links['b'])]
abset = bike_links.groupby('abset')[['a']].agg(len)
oneway_abset = abset[abset['a']>1].index.values

bike_links['oneway'] = False
bike_links.loc[bike_links['abset'].isin(oneway_abset),'oneway'] = True

oneway_dict = bike_links['oneway'].to_dict()
links.loc[links['cycleway']!='no','oneway'] = bike_links['oneway']

In [556]:
links = fill_na_col(links,'highway','maxspeed',np.mean)


KeyError: 'Column not found: maxspeed'

In [557]:
links

,id,type,highway,speed,lanes,name,oneway,surface,cycleway,a,b,geometry,combine_cycle_tag,length,time,incline
road_link_0,4320471,way,residential,30.0,1.0,Rue Mackay,True,asphalt,no,road_node_8702,road_node_8772,"LINESTRING (-73.57878 45.49667, -73.57870 45.4...",no no no no,7.838556,0.940627,-7.270209
road_link_1,4320487,way,residential,20.0,1.0,Rue Saint-Paul Est,True,sett,no,road_node_2893,road_node_21755,"LINESTRING (-73.55198 45.50863, -73.55200 45.5...",no no no no,46.471605,8.364889,-1.232730
road_link_2,10530798,way,residential,30.0,1.0,Rue Saint-Ambroise,True,asphalt,no,road_node_24396,road_node_21124,"LINESTRING (-73.57689 45.47829, -73.57693 45.4...",no no no no,27.685876,3.322305,0.000000
road_link_3,20411147,way,residential,40.0,2.0,Avenue Beverley,True,asphalt,no,road_node_1867,road_node_9413,"LINESTRING (-73.64193 45.52234, -73.64158 45.5...",no no no no,69.016711,6.211504,0.000000
road_link_4,408865340,way,tertiary,40.0,2.0,Chemin Canora,True,asphalt,no,road_node_18193,road_node_18564,"LINESTRING (-73.64970 45.51853, -73.65063 45.5...",no no no no,78.552479,7.069723,1.458475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
road_link_97820,"[1077987400, 1077987406]",way,secondary_link,50.0,2.0,NaN,True,asphalt,yes,road_node_12928,road_node_8471,"LINESTRING (-73.59752 45.47296, -73.59759 45.4...",no no no yes,15.921584,1.146354,3.593902
road_link_97821,"[205590924, 963708573, 1177247429, 963708572]",way,secondary_link,50.0,2.0,NaN,True,asphalt,no,road_node_18668,road_node_10175,"LINESTRING (-73.62829 45.55079, -73.62834 45.5...",no no no no,47.595305,3.426862,-1.203634
road_link_97822,"[471381659, 963708570, 963708571]",way,secondary_link,50.0,2.0,NaN,True,asphalt,no,road_node_21630,road_node_24809,"LINESTRING (-73.62772 45.55162, -73.62762 45.5...",no no no no,45.170145,3.252250,0.000000
road_link_97823,"[1077987398, 1077987400]",way,secondary_link,50.0,2.0,NaN,True,asphalt,yes,road_node_8471,road_node_27511,"LINESTRING (-73.59768 45.47305, -73.59770 45.4...",no no no yes,61.416212,4.421967,3.726376


In [314]:
'''
links = rename_bicycle_tags(links,'oneway:bicycle')
    links = rename_bicycle_tags(links,'cycleway:left:oneway')
    links = rename_bicycle_tags(links,'cycleway:right:oneway')

    links['combine_oneway_tag'] = links['oneway:bicycle'] +'-'+ \
                                    links['cycleway:left:oneway']+'-'+ \
                                    links['cycleway:right:oneway'] 
'''

"\nlinks = rename_bicycle_tags(links,'oneway:bicycle')\n    links = rename_bicycle_tags(links,'cycleway:left:oneway')\n    links = rename_bicycle_tags(links,'cycleway:right:oneway')\n\n    links['combine_oneway_tag'] = links['oneway:bicycle'] +'-'+                                     links['cycleway:left:oneway']+'-'+                                     links['cycleway:right:oneway'] \n"

In [541]:
links = road.clean_oneway(links)

In [542]:
links = road.clean_maxspeed(links)

In [543]:
links = road.drop_duplicated_links(links)

764 links dropped


In [544]:
links = road.simplify(links,cutoff=10)
len(links)

20985 deg 2 nodes
find path with large cutoff for 1  origins
find path with large cutoff for 1  origins
find path with large cutoff for 1  origins
find path with large cutoff for 2  origins
0 links were not merge because the oneway field is not the same
0 links were not merge because the highway field is not the same
0 merged_links unmerged because the geometry became a multilinestring


55071

In [545]:
links = road.split_oneway(links)
len(links)

98782

In [546]:
links = road.main_strongly_connected_component(links,None,False)
len(links)

97825

In [547]:
links['maxspeed'] = links['maxspeed'].apply(lambda x: process_list_in_col(x,float,np.nanmean))
links['lanes'] = links['lanes'].apply(lambda x: process_list_in_col(x,float,lambda x: np.floor(np.nanmean(x))))

../road.py:24: RuntimeWarning: Mean of empty slice
  return  function([new_type(val) for val in col_values])


In [548]:
links['cycleway'] = links['cycleway'].apply(lambda x: process_list_in_col(x,str,lambda x: np.sort(x)[-1]))

In [549]:
for col in ['highway','name','surface']:
    links[col] = links[col].apply(lambda x: remove_list_in_col(x,'first'))

In [550]:
links = fill_na_col(links,'highway','maxspeed',np.mean)
links = fill_na_col(links,'highway','lanes',lambda x: np.floor(np.mean(x)))

In [551]:

# Add length
print("Write Links and Nodes ...")
epsg = get_epsg(nodes.iloc[0]['geometry'].y, nodes.iloc[0]['geometry'].x)
links['length'] = links.to_crs(epsg).length

# Add Time
links['time'] = links['length']/(links['maxspeed']*1000/3600)
links = links.rename(columns = {'maxspeed' : 'speed'})

# reindex and remove ununsed nodes
links = links.reset_index(drop=True)
links.index = 'road_link_'+links.index.astype(str)
nodes_set = set(links['a']).union(set(links['b']))
nodes = nodes.loc[list(nodes_set)].sort_index()

Write Links and Nodes ...


In [552]:
print('Adding elevation')
el_dict = get_elevation_from_srtm(nodes)
nodes['elevation'] = nodes.index.map(el_dict.get)
# incline from node a to b in deg. neg if going down (if b is lower dans a)
links['incline'] = calc_incline(links['a'].apply(lambda x: el_dict.get(x)).values,
                            links['b'].apply(lambda x: el_dict.get(x)).values,
                            links['length'].values)


Adding elevation
file save to /tmp


In [554]:
links.to_file('road_links.geojson')
nodes.to_file('road_nodes.geojson')

ValueError: Invalid field type <class 'list'>

In [16]:
from road import *
from bike import *
from elevation import *

In [19]:
links, nodes = get_links_and_nodes('way.geojson', split_direction=False)
nodes = nodes.set_crs(links.crs)
links = test_bicycle_process(links,cycleway_columns,highway_list)
links = clean_oneway(links)
links = clean_maxspeed(links)
links = drop_duplicated_links(links, sort_column='maxspeed')
links = simplify(links)
links = split_oneway(links)
links = main_strongly_connected_component(links,None,False)

print('removing list in columns ...')
links['maxspeed'] = links['maxspeed'].apply(lambda x: process_list_in_col(x,float,np.nanmean))
links['lanes'] = links['lanes'].apply(lambda x: process_list_in_col(x,float,lambda x: np.floor(np.nanmean(x))))
if 'cycleway' in links.columns:
    links['cycleway'] = links['cycleway'].apply(lambda x: process_list_in_col(x,str,lambda x: np.sort(x)[-1]))
for col in ['id', 'type', 'highway','name','surface']:
    links[col] = links[col].apply(lambda x: remove_list_in_col(x,'first'))


# Fill NaN with mean values by highway
links = fill_na_col(links, 'highway', 'maxspeed', np.mean)
links = fill_na_col(links, 'highway', 'lanes', lambda x: np.floor(np.mean(x)))

# Add length
print("Write Links and Nodes ...")
epsg = get_epsg(nodes.iloc[0]['geometry'].y, nodes.iloc[0]['geometry'].x)
links['length'] = links.to_crs(epsg).length

# Add Time
links['time'] = links['length']/(links['maxspeed']*1000/3600)
links = links.rename(columns = {'maxspeed' : 'speed'})

# reindex and remove ununsed nodes
links = links.reset_index(drop=True)
links.index = 'road_link_'+links.index.astype(str)
nodes_set = set(links['a']).union(set(links['b']))
nodes = nodes.loc[list(nodes_set)].sort_index()


['no', 'yes', 'shared']
['no', 'yes', 'shared']
['no', 'yes', 'shared']
['no', 'yes', 'shared']
182 links dropped
18297 deg 2 nodes
find path with large cutoff for 2  origins
find path with large cutoff for 1  origins
find path with large cutoff for 1  origins
find path with large cutoff for 2  origins
find path with large cutoff for 6  origins
0 links were not merge because the oneway field is not the same
0 links were not merge because the highway field is not the same
0 merged_links unmerged because the geometry became a multilinestring
removing list in columns ...


../road.py:24: RuntimeWarning: Mean of empty slice
  return  function([new_type(val) for val in col_values])


Write Links and Nodes ...


In [20]:
print('Adding elevation')
el_dict = get_elevation_from_srtm(nodes)
nodes['elevation'] = nodes.index.map(el_dict.get)
# incline from node a to b in deg. neg if going down (if b is lower dans a)
links['incline'] = calc_incline(links['a'].apply(lambda x: el_dict.get(x)).values,
                            links['b'].apply(lambda x: el_dict.get(x)).values,
                            links['length'].values)

Adding elevation
file save to /tmp


In [26]:
links.to_file('road_links.geojson')
nodes.to_file('road_nodes.geojson')

In [25]:
links = links.drop(columns='combine_cycle_tag')